### A markov-chain sentence generator

There's some cruft here (like the bit to not take sentences with "PERSON", "LOCATION", "ORGANIZATION").  These things happen.

In [3]:
import codecs, re

def decruft_sentences(path_to_file):

    sentences = []
    
    lines = codecs.open(path_to_file, 'r', encoding='utf-8').read().split('\n')
    
    print 'len(lines)', len(lines)
    
    for line in lines:
        if line.strip() > '':
            cliche = line.strip().split('|')[0]
            sentence = line.strip().split('|')[1]
            
            sentence = re.sub('\* ', '', sentence)
            sentence = re.sub('\"', '', sentence)
            sentence = re.sub('\[Illustration\]', '', sentence)
            
            drop_this_one = False
            if 'chapter' in sentence.lower() or '..............' in sentence.lower():
                drop_this_one = True
            elif '[' in sentence.lower():
                drop_this_one = True
            
            if drop_this_one == False:
                
                tokens = re.split('[^A-Z]', sentence)
                for t in tokens:
                    if len(t) > 1 and t.upper() == t and t not in ['PERSON', 'LOCATION', 'ORGANIZATION']:
                        drop_this_one = True
                        break
            
            if drop_this_one == False:
                
                sentences.append(re.sub('\s+', ' ', sentence.strip()))

    print 'len(sentences)', len(sentences)
    
    return sentences

original_sentences = decruft_sentences('1_selected_sentences.UNIQ.txt')

len(lines) 16136
len(sentences) 15743


In [4]:
import codecs, re
from collections import defaultdict

KEY_LENGTH = 2

original_sentences = decruft_sentences('1_selected_sentences.UNIQ.txt')

markov_chain = defaultdict(lambda : defaultdict(int))

for s in original_sentences:
    
    tokens = ['START_SENTENCE']
    
    cleaned_s = s.replace('"', '').replace('[', '').replace(']', '').replace('*', ' ')
    cleaned_s = re.sub('\s+', ' ', cleaned_s.strip())
    
    for t in re.split('\s+', cleaned_s):
        if t > '':
            tokens.append(t)
    tokens.append('END_SENTENCE')
    
    for a in range(0, len(tokens) - KEY_LENGTH):
        markov_chain[tuple(tokens[a: a + KEY_LENGTH])][tokens[a + KEY_LENGTH]] += 1

starting_points = []
for k in markov_chain:
    if k[0] == 'START_SENTENCE':
        starting_points.append(k)
        
original_sentences = set(original_sentences)

print 'len(original_sentences)', len(original_sentences)
print 'len(markov_chain)', len(markov_chain)
print 'len(starting_points)', len(starting_points)


len(lines) 16136
len(sentences) 15743
len(original_sentences) 15534
len(markov_chain) 194986
len(starting_points) 2134


In [12]:
import random

NUMBER_OF_SENTENCES_TO_CREATE = 10000

new_sentences = []

while len(new_sentences) < NUMBER_OF_SENTENCES_TO_CREATE:

    markov_key = random.sample(starting_points, 1)[0]

    result_tokens = [markov_key[1]]

    n_times_in_token_loop = 0
    token_loop_okay = True

    while True:

        # I DON'T WANT TO ENDLESSLY CIRCLE IN THIS LOOP
        n_times_in_token_loop += 1
        if n_times_in_token_loop > 500:
            token_loop_okay = False
            break

        possible_next_tokens = []
        for token, occurrences in markov_chain[markov_key].iteritems():
            for a in range(0, occurrences):
                possible_next_tokens.append(token)

        next_token = random.sample(possible_next_tokens, 1)[0]

        if next_token == 'END_SENTENCE':
            break
        else:
            result_tokens.append(next_token)

        markov_key = tuple(list(markov_key[1:]) + [next_token])

    result_sentence = ' '.join(result_tokens)

    if token_loop_okay == True and \
        result_sentence not in original_sentences and \
        result_sentence not in new_sentences:
        
        new_sentences.append(result_sentence)

f = codecs.open('2_markov_sentences.txt', 'w', encoding='utf-8')
for s in new_sentences:
    f.write(s + '\n')
f.close()

print 'Done!'

Done!


In [11]:
!wc -l 2_markov_sentences.txt
!wc -w 2_markov_sentences.txt

2500 2_markov_sentences.txt
65050 2_markov_sentences.txt
